# Assignment - Segmenting and Clustering Neighborhoods in Toronto

name: David Lu

In [1]:
import pandas as pd

In [6]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(f'Total tables: {len(tables)}')

Total tables: 3


In [28]:
df = tables[0]
df.rename(
    columns = {'Postal Code': 'PostalCode', 'Borough': 'Borough', 'Neighbourhood': 'Neighborhood'},
    inplace = True)
print(df.shape)
df.head()

(180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Checking DataFrame is Created Correctly

#### The dataframe consist of three columns: PostalCode, Borough, and Neighborhood

In [30]:
assert df.columns.to_list() == ['PostalCode', 'Borough', 'Neighborhood']

#### Remove all entries where Borough is "Not assigned"

In [34]:
num_notassigned_in_borough = (df.Borough == 'Not assigned').sum()
print(f"There are {num_notassigned_in_borough} 'Not assigned' in Borough")

There are 77 'Not assigned' in Borough


In [31]:
df_ = df[df.Borough != 'Not assigned']
print(df_.shape)
df_.head()

(103, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Make sure there are no repeated Postal Code

In [39]:
if df_.PostalCode.nunique() == df_.shape[0]:
    print("There are no repeated Postal Code")
else:
    print("There are repeated Postal Code, and we must clean them such that they're listed in Neighborhood")

There are no repeated Postal Code


### Check if there are any borough but Not assigned neighborhood

In [48]:
if not (df_['Neighborhood'] == 'Not assigned').values.all():
    print(f"There are no 'Not assigned' value in the 'Neighborhood' column of the DataFrame.")
else:
    print(f"Must clean the DataFrame accordingly")

There are no 'Not assigned' value in the 'Neighborhood' column of the DataFrame.


### Print out the shape of the DataFrame

In [42]:
print(f"The shape of the DataFrame is {df_.shape}.")

The shape of the DataFrame is (103, 3).
